In [3]:
#-----------------------------------------pre-processing---------------------------------------------------#

#data loading
import pandas as pd
predata = pd.read_csv('traindata.csv', nrows = 400000) #aumentar nrows até rebentar

In [4]:
#type(predata['ipaddress'])

In [5]:
#set(predata['cons_freq_ipaddress'])

In [6]:
#len(set(predata['cons_freq_ipaddress']))

In [7]:
#delete unnecessary columns
predata.drop(predata.columns[0], axis = 1, inplace = True) 

predata.drop(['ID', 'rowid', 'clientid', 'client', 'fingerprint', 'description_originator', 
'description_beneficiary', 'dummy_var', '#fingerprint_30D', 'fingerprint_time_diff', 'clientid_ipaddress_30D', 
'clientid_fingerprint_30D', 'clientid_iban_orig_30D', 'clientid_iban_dest_30D', '#clientid_fingerprint_30D',
'amount_ratio_avg_clientid_30D', 'amount_ratio_avg_iban_dest_30D', 'canal__MBE', 'canal__MBP', 'canal__NBE', 
'canal__NBP', 'canal__OBE', 'operativa__PAGSRV', 'operativa__TRFINT', 'operativa__TRFIPS', 'operativa__TRFITC', 
'operativa__TRFSEP', 'operativa__others', 'entity__3', 'entity__37','entity__469', 'entity__5623', 'entity__others', 
'browser_family__chrome', 'browser_family__chrome mobile', 'browser_family__edge', 'browser_family__firefox', 
'browser_family__ie', 'browser_family__mobile safari', 'browser_family__others', 'os_family__android', 'os_family__ios', 
'os_family__linux', 'os_family__mac os x', 'os_family__others', 'os_family__windows', 'trusted_indicator__0.0',
'trusted_indicator__1.0', 'trusted_indicator__UNKNOWN', 'week', 'amount_ratio_avg_30D_per_clientid',
'amount_ratio_max_30D_per_clientid', 'amount_over_account_balance', 'amount_is_integer', 'amount_categories',
'cons_freq_fingerprint', 'cons_freq_clientid', 'cons_time_fingerprint'], axis = 1, inplace = True)

In [8]:
#columns created by me
predata['weekday'] = pd.to_datetime(predata['timestamp']).apply(lambda x: x.weekday())

In [9]:
#stopped being necessary
predata.drop(['timestamp'], axis = 1, inplace = True)

In [10]:
#remove dots from ipaddress
predata['ipaddress'] = predata['ipaddress'].apply(lambda x: "".join(x.split(".")))

In [11]:
#convert decimals to int 
cols_to_int = ['#clientid_30D', '#ipaddress_30D', '#iban_orig_30D', '#iban_dest_30D', 'clientid_time_diff',
               'iban_dest_time_diff', '#clientid_ipaddress_30D', '#clientid_iban_orig_30D', '#clientid_iban_dest_30D']

for col in cols_to_int:
    predata[col] = predata[col].apply(lambda x: int(x))

In [12]:
#add letter before number to distinguish between similar words from different columns
cols = ['entity', 'reference', 'iban_orig', 'iban_dest', 'ipaddress', 'hour', 'is_fraud_cons', 'date', 
'cons_freq_ipaddress', '#clientid_30D', '#ipaddress_30D', '#iban_orig_30D', '#iban_dest_30D', 'clientid_time_diff', 
'iban_dest_time_diff', '#clientid_ipaddress_30D', '#clientid_iban_orig_30D', '#clientid_iban_dest_30D']

identifier = ['e', 'r', 'io', 'id', 'ip', 'h', 'fc', 'd', 'cfi', 'ncd', 'nid', 'niod', 'nidd', 'ctd', 'idtd', 'ncid', 
'nciod', 'ncidd']

for col in range(len(cols)):
    predata[cols[col]] = predata[cols[col]].apply(lambda x: identifier[col] + str(x))

In [13]:
#convert binary and UNKNOWN to words 
def apply_map(df, col, target_map):
    df[col] = df[col].apply(lambda x: target_map.get(str(x)))

cols_maps = [('is_fraud', {'0':'nfraud', '1':'fraud'}),
             ('trusted_indicator', {'0.0':'ntrusted', '1.0':'trusted', 'UNKNOWN':'unknown'}),
             ('is_mobile', {'False':'nmobile', 'True':'mobile'}),
             ('is_tablet', {'False':'ntablet', 'True':'tablet'}),
             ('is_pc', {'False':'npc', 'True':'pc'}),
             ('is_touch', {'False':'ntouch', 'True':'touch'}),
             ('is_bot', {'False':'nbot', 'True':'bot'}),
             ('weekday', {'0':'seg', '1':'ter', '2':'qua', '3':'qui', '4':'sex', '5':'sab', '6':'dom'})]

for comb in cols_maps:
    apply_map(predata, comb[0], comb[1])

In [14]:
#how many frauds
len(predata[predata['is_fraud'] == 'fraud'])

3

In [15]:
#how many not frauds
len(predata[predata['is_fraud'] == 'nfraud'])

399997

In [16]:
#amounts -> classes
import numpy as np 

#automatic labels
import string

class LabelCategorizer():
    def __init__(self, base_word = 'cat'):
        self._alphabet_index = 0
        self.base_word = base_word
        self.current_word = self.base_word
        self.shift = 0

    def __str__(self):
        return 'Class: Label Categorizer\nBase word: ' + self.base_word + '\nCurrent Word: ' + self.current_word

    def get_next_word(self):
        if self.shift > 0:
            self.current_word = self.current_word[-1] + self.current_word[:-1]
            self.shift -= 1
        else:
            self.current_word = self.current_word + string.ascii_lowercase[self._alphabet_index]
            self._alphabet_index = (self._alphabet_index + 1) % len(string.ascii_lowercase)
            self.shift = len(self.current_word) - 1

        return self.current_word


#replacement of the old columns with the new ones with classes 
def cutter (col, number, word, words_map):

    bins_a = np.linspace(predata[col].min(), predata[col].max(), num = number)
    bins_a[0] = bins_a[0]-1
    bins_aux = bins_a[1:]
    bins_aux = np.append(bins_aux, bins_a[-1]+1)
    bin_tuples = list(zip(bins_a, bins_aux))

    bins = pd.IntervalIndex.from_tuples(bin_tuples)

    labels_a = []

    a = LabelCategorizer(base_word = word)

    for _ in range(number):
        labels_a.append(a.get_next_word())

    x = pd.cut(predata[col].to_list(), bins = bins)
    x.categories = labels_a
    predata[col] = x

    
    for i in range(number):
        words_map[labels_a[i]] = bins[i]       

        
columns = ['amount', 'accountbalance', 'mean_amount_clientid_30D', 'mean_amount_iban_dest_30D']
number_bins = [100, 250, 100, 100]
base_words = ['blue', 'pink', 'red', 'grey']

#classes name -> interval
values_map = {}

for i in range (len(columns)):
    cutter(columns[i], number_bins[i], base_words[i], values_map)

In [17]:
#values_map

In [18]:
#values_map['pinka']

In [19]:
#len(predata.columns)

In [20]:
#predata.columns

In [21]:
#lower case
predata['canal'] = predata['canal'].str.lower()
predata['operativa'] = predata['operativa'].str.lower()
predata['browser_family'] = predata['browser_family'].str.lower()
predata['os_family'] = predata['os_family'].str.lower()
predata['ipaddress'] = predata['ipaddress'].str.lower()

In [22]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#predata

In [23]:
#dataframe to numpy array of arrays
sentences = predata.to_numpy()

In [24]:
#type(sentences)

In [25]:
sentences[0]

array(['nbe', 'apgest', 'e3', 'r542046', 'ntrusted', 'io4', 'id3',
       'bluea', 'nkapi', 'ip17218224250', 'nfraud', 'nmobile', 'ntablet',
       'pc', 'ntouch', 'nbot', 'chrome', 'windows', 'ncd0', 'nid1056',
       'niod0', 'nidd56720', 'ctd-1', 'idtd4', 'reda', 'greya', 'ncid0',
       'nciod0', 'ncidd0', 'fc0', 'd20190806', 'h0', 'cfi1', 'ter'],
      dtype=object)

In [26]:
#checks = [type(elem) == str for elem in sentences[0]]
#sum(checks) == len(sentences[0])

In [27]:
#type(sentences[99][0]) == str

In [28]:
#len(sentences[0])

In [29]:
#sentences = list(sentences)
#np.savetxt('train_data.txt', sentences, delimiter=" ", fmt="%s")
#sentences = w2v.LineSentence('train_data.txt')

In [30]:
sentences_aux = [list(curr) for curr in sentences]

In [31]:
#sentences_aux[0]

In [32]:
sentences_series = pd.Series(sentences_aux)

In [33]:
#callback to print loss after each epoch

from gensim.models.callbacks import CallbackAny2Vec
import gensim.models.word2vec as w2v

# init callback class
class callback(CallbackAny2Vec):
    
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
#---------------------------------------------------training------------------------------------------------------#

#inicialization and training word2vec
import multiprocessing
import gensim.models.word2vec as w2v

def training (sentences, cycles, dim, architecture, context):
    model = w2v.Word2Vec (
        sg = architecture, #1 - skip-gram , 0 - cbow
        workers = multiprocessing.cpu_count(), #uses all the cores 
        vector_size = dim, #dimension of the embedding space = N 
        window = context, #words befores and after center word
        sample = 0, #whithout downsampling 
        ns_exponent = 0.0,
        min_count = 0, 
)
    
    #vocabulary creation
    model.build_vocab(sentences) 

    #model training
    model.train(sentences, epochs = cycles, total_examples = model.corpus_count, compute_loss = True, callbacks = [callback()])

    return model

#model creation
model = training(sentences_series, 5, 5, 1, 5) #usar 2, 5, 7, 10 para espaço do meio

Loss after epoch 0: 5944069.0


In [ ]:
print(model)

In [ ]:
print(len(model.wv.index_to_key)) #vocab size = V

In [ ]:
#print(model.wv.index_to_key)

In [ ]:
#words more related to not frauds
#model.wv.most_similar('nfraud')

In [ ]:
#saving the model
model.save('Trained.w2v')

In [ ]:
#-------------------------------------------------matrices--------------------------------------------------------#

#weight matrices
m1 = model.wv.vectors  #input embedding matrix  #VxN

In [ ]:
m2 = model.syn1neg      #NxV 

In [ ]:
#visualization of the matrices
#np.set_printoptions(edgeitems = 10000)
#np.set_printoptions(linewidth = 10000)
np.set_printoptions(threshold = np.inf)

#import sys
#np.set_printoptions(threshold = sys.maxsize)

#seeing the matrix
#s = [[str(e) for e in row] for row in probabilities]
#lens = [max(map(len, col)) for col in zip(*s)]
#fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
#table = [fmt.format(*row) for row in s]
#print ('\n'.join(table))

In [ ]:
print(m1.shape)
#print(np.matrix(m1))

In [ ]:
print(m2.shape)
#print(np.matrix(m2))

In [ ]:
m3 = np.transpose(m2)
print(m3.shape)
#print(np.matrix(m3))

In [ ]:
#product of the matrices
probabilities = np.matmul(m1, m3)
print(probabilities.shape)
#print(np.matrix(probabilities))

In [ ]:
#conditional probabilities - softmax
#def softmax(x):
#    y = np.exp(x - np.max(x))
#    f_x = y / np.sum(np.exp(x))
#    return f_x

#all_probs = [max(softmax(curr)) for curr in probabilities]

from scipy.special import softmax

all_probs = softmax(probabilities)

In [ ]:
np.max(all_probs)

In [ ]:
np.min(all_probs)

In [ ]:
all_probs.sum()

In [ ]:
#check if there's negative numbers

#for number in conditional_probabilities:
#    if np.any(number) < 0:
#        print('Something is wrong')
#    else:
#        print('All positive')

In [ ]:
#check if there numbers bigger than 1

#for number in conditional_probabilities:
#    if np.any(number) > 1:
#        print('Something is wrong')
#    else:
#        print('All positive')

In [ ]:
#-----------------------------------------------------LDA model----------------------------------------------------------#

import gensim
import gensim.corpora as corpora

#create dictionary
id2word = corpora.Dictionary(sentences)

#create corpus
texts = sentences

#term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

#human readable format of corpus (term-frequency)
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]) #so esta a fazer para a 1ª linha

#build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = id2word,
                                           num_topics = 20,
                                           random_state = 100,
                                           update_every = 1,
                                           chunksize = 100,
                                           passes = 100,
                                           alpha = 'auto',
                                           per_word_topics = True)


doc_lda = lda_model[corpus]

In [ ]:
lda_model.print_topics()

In [ ]:
#compute perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  #the lower the better.

#compute coherence score                                 #higher the topic is more human interpretable
coherence_model_lda = CoherenceModel(model = lda_model, texts = sentences, dictionary = id2word, coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
import pyLDAvis.gensim_models

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

In [ ]:
vis

In [ ]:
id2word[20]